In [1]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import pynq.lib.dma
import time

In [2]:
weight_0 = np.load('weights/weight_0.npy')
weight_1 = np.load('weights/weight_1.npy')
weight_2 = np.load('weights/weight_2.npy')
weight_3 = np.load('weights/weight_3.npy')
bias_0 = np.load('weights/bias_0.npy')
bias_1 = np.load('weights/bias_1.npy')
bias_2 = np.load('weights/bias_2.npy')
bias_3 = np.load('weights/bias_3.npy')

In [3]:
test = np.loadtxt("dataset/test.csv", delimiter=',', skiprows=1, usecols=np.arange(0,561))

In [45]:
test_size = 1000
np.random.seed(2)
a = np.arange(2000)
np.random.shuffle(a)
a = a[:test_size]
testset = test[a]
testset.shape

(1000, 561)

In [29]:
overlay = Overlay('bitstream.bit')   # load bitstream inside FPGA
dma = overlay.axi_dma_0         # define module DMA

In [30]:
input_buffer0 = allocate(shape=(561,), dtype=np.float32)
input_buffer1 = allocate(shape=(64,), dtype=np.float32)
input_buffer2 = allocate(shape=(6,), dtype=np.float32)
output_buffer0 = allocate(shape=(6,), dtype=np.float32)

In [31]:
##weights
for i in range(64):
    for k in range(561):
        input_buffer0[k] = weight_0[k][i]
    dma.sendchannel.transfer(input_buffer0)
    dma.sendchannel.wait()
for i in range(64):
    for k in range(64):
        input_buffer1[k] = weight_1[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
for i in range(64):
    for k in range(64):
        input_buffer1[k] = weight_2[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
for i in range(6):
    for k in range(64):
        input_buffer1[k] = weight_3[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
##bias
for k in range(64):
    input_buffer1[k] = bias_0[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(64):
    input_buffer1[k] = bias_1[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(64):
    input_buffer1[k] = bias_2[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(6):
    input_buffer2[k] = bias_3[k]
dma.sendchannel.transfer(input_buffer2)
dma.sendchannel.wait()

In [46]:
result = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
timestart = time.time()
for i in range(test_size):
    for j in range(561):
        input_buffer0[j] = testset[i][j];
    dma.sendchannel.transfer(input_buffer0)
    dma.sendchannel.wait()
    dma.recvchannel.transfer(output_buffer0)
    dma.recvchannel.wait()
    result = np.row_stack((result, output_buffer0))
timeend = time.time()

In [47]:
np.argmax(result[1:], axis=-1)

array([4, 1, 0, 3, 4, 3, 5, 0, 2, 3, 3, 1, 3, 0, 0, 4, 3, 3, 3, 4, 2, 5,
       0, 1, 0, 2, 1, 1, 2, 5, 0, 0, 1, 2, 0, 0, 5, 1, 2, 5, 3, 1, 4, 1,
       0, 3, 0, 1, 1, 0, 2, 1, 3, 2, 0, 5, 4, 1, 3, 2, 5, 4, 2, 0, 5, 4,
       4, 2, 1, 0, 1, 2, 1, 3, 4, 5, 0, 5, 3, 1, 3, 4, 4, 3, 3, 3, 3, 0,
       1, 4, 3, 2, 2, 1, 3, 5, 5, 2, 2, 1, 3, 0, 0, 3, 5, 1, 3, 2, 0, 5,
       5, 1, 1, 2, 5, 3, 3, 5, 4, 2, 2, 3, 1, 4, 4, 4, 5, 0, 0, 3, 4, 5,
       5, 1, 4, 3, 1, 0, 0, 2, 1, 5, 3, 0, 2, 2, 1, 1, 5, 2, 3, 2, 3, 3,
       0, 3, 2, 1, 3, 3, 4, 5, 2, 2, 4, 0, 3, 5, 4, 0, 4, 2, 3, 2, 1, 1,
       1, 1, 0, 5, 0, 0, 5, 5, 5, 5, 5, 0, 1, 3, 5, 5, 1, 0, 0, 2, 4, 1,
       5, 0, 1, 5, 5, 2, 0, 5, 5, 0, 2, 5, 4, 2, 4, 4, 0, 0, 3, 4, 0, 1,
       0, 3, 2, 4, 5, 4, 2, 1, 2, 3, 0, 0, 0, 2, 0, 3, 1, 3, 2, 2, 2, 4,
       5, 1, 3, 3, 0, 5, 3, 0, 4, 0, 4, 0, 1, 4, 2, 0, 3, 2, 0, 4, 5, 1,
       3, 0, 3, 5, 2, 0, 5, 3, 2, 3, 5, 4, 5, 1, 2, 1, 3, 1, 2, 3, 2, 3,
       2, 0, 4, 1, 4, 4, 0, 0, 2, 5, 1, 4, 3, 2, 1,

In [48]:
print('Time taken = ' + str(timeend - timestart))

Time taken = 5.5103535652160645
